In [ ]:
# 00_acs-merge.ipynb
# Initial Cleaning and Standardization of ACS data from 2013 to 2013 for various tables

# Libraries
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import glob
from pathlib import Path

sys.path.append(str(Path().resolve().parent / "src"))
from cleaning_functions import preview_df

# Paths
DATA_DIR = Path("../data/")
ACS_DIR = DATA_DIR / "acs"
TRACTS_PATH = DATA_DIR / "tl_2020_36_tract/tl_2020_36_tract.shp"

In [ ]:
## --- FUNCTION TO LOAD DATA ---

# Load ACS Data
def load_clean_acs(folder, col_map, topic):
    """
    Loads and cleans ACS data from a specified folder.

    Parameters
    ----------
    folder : str or Path
        Path to the folder containing ACS CSV files.
    col_map : dict
        Dictionary mapping original ACS column names to new column names.
    topic : str
        Topic of the ACS data (e.g., 'demographics', 'housing').

    Returns
    -------
    pd.DataFrame
        Concatenated and cleaned DataFrame with selected and renamed columns, including 'GEOID' and 'year'.
    """
    all_files = sorted(glob.glob(os.path.join(folder, "*-Data.csv")))
    dfs = []

    for file in all_files:
        # Extract year from filename (assuming contains '20XX')
        filename = os.path.basename(file)
        match = re.search(r"20\d{2}", filename)
        if match:
            year = match.group()
        else:
            raise ValueError(f"Year not found in filename: {filename}")

        print(f"Loading {topic} data for {year} from {file}")

        df = pd.read_csv(file, dtype=str)
        df.columns = df.columns.str.strip()

        # Add year column
        df["year"] = int(year)

        df['GEOID'] = df['GEO_ID']

        # Select relevant columns and rename
        cols_to_keep = ["GEOID", "year"] + list(col_map.keys())
        df_clean = df[cols_to_keep]
        df_clean = df_clean.rename(columns=col_map)

        for col in col_map.values():
            df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

        dfs.append(df_clean)

    return pd.concat(dfs, ignore_index=True)


In [ ]:
## --- DEFINE TABLES AND VARIABLES FOR INPUT ---
# Define mappings per table
rent_cols = {"B25077_001E": "median_rent"}
pop_cols = {"B01003_001E": "population"}
poverty_cols = {"B17017_001E": "poverty_total", "B17017_002E": "poverty_below"}
race_cols = {
    "B02001_001E": "race_total",
    "B02001_002E": "race_white",
    "B02001_003E": "race_black",
    "B02001_004E": "race_native",
    "B02001_005E": "race_asian",
    "B02001_006E": "race_pi",
    "B02001_007E": "race_other",
    "B02001_008E": "race_mixed",
}

In [ ]:
## --- LOAD AND PREVIEW DATA ---
# ---- Load latest available data ----
rent = load_clean_acs(
    folder=os.path.join(ACS_DIR, "ACS_median-rent"),
    col_map=rent_cols,
    topic="median_rent",
)
pop = load_clean_acs(
    folder=os.path.join(ACS_DIR, "ACS_population"), col_map=pop_cols, topic="population"
)
poverty = load_clean_acs(
    folder=os.path.join(ACS_DIR, "ACS_poverty"), col_map=poverty_cols, topic="poverty"
)
race = load_clean_acs(
    folder=os.path.join(ACS_DIR, "ACS_race"), col_map=race_cols, topic="race"
)

print(rent.columns)
print(pop.columns)
print(poverty.columns)
print(race.columns)

In [ ]:
## --- IMPORT EDUCATION TABLE (special case) ---
# Define education columns mapping for aggregation
education_groups = {
    "education_no_diploma": [
        "B15003_002E",
        "B15003_003E",
        "B15003_004E",
        "B15003_005E",
        "B15003_006E",
        "B15003_007E",
        "B15003_008E",
        "B15003_009E",
        "B15003_010E",
        "B15003_011E",
        "B15003_012E",
        "B15003_013E",
        "B15003_014E",
        "B15003_015E",
        "B15003_016E",
    ],
    "education_high_school": [
        "B15003_017E",
        "B15003_018E",
        "B15003_019E",
        "B15003_020E",
    ],
    "education_associates": ["B15003_021E"],
    "education_bachelors": ["B15003_022E"],
    "education_masters": ["B15003_023E"],
    "education_professional": ["B15003_024E"],
    "education_doctorate": ["B15003_025E"],
}

# Flatten the mapping for column import
education_cols = {col: col for group in education_groups.values() for col in group}

# Load and aggregate the education data
education_raw = load_clean_acs(
    folder=os.path.join(ACS_DIR, "ACS_education"),
    col_map=education_cols,
    topic="education",
)

# Aggregate the columns into categories
education = education_raw.copy()
for new_col, old_cols in education_groups.items():
    education[new_col] = education[old_cols].sum(axis=1)

# Keep only GEOID, year, and new education summary columns
education = education[["GEOID", "year"] + list(education_groups.keys())]

print(education.columns)

In [ ]:
## --- MERGE DATA ---
# Merge on geoid 
acs_merged = (
    rent.merge(pop, on=["GEOID", "year"], how="inner")
    .merge(poverty, on=["GEOID", "year"], how="inner")
    .merge(race, on=["GEOID", "year"], how="inner")
    .merge(education, on=["GEOID", "year"], how="inner")
)
acs_merged = acs_merged[acs_merged["GEOID"] != "Geography"]

preview_df(acs_merged)
print(acs_merged.head())
print(acs_merged.dtypes)

print(acs_merged["GEOID"].head(20))

In [ ]:
## --- ADD CENSUS TRACTS GEO DATA ---
# Load NYC Census Tracts
gdf_tracts = gpd.read_file(TRACTS_PATH)
gdf_tracts = gdf_tracts.to_crs(epsg=2263)

# Clean ACS GEOIDs to match shapefile format (drop prefix)
acs_merged["GEOID"] = acs_merged["GEOID"].str.extract(r"(\d{11})")

# Merge ACS data into shapefile (geometry-aware)
gdf_acs = gdf_tracts[["GEOID", "geometry"]].merge(acs_merged, on="GEOID", how="right")

# Drop NA values in essential columns
gdf_acs = gdf_acs.dropna(subset=["geometry", "median_rent"])

# Check for geometry loss or mismatches
print(gdf_acs["geometry"].isna().mean())  
preview_df(gdf_acs)

In [ ]:
## --- CONVERT GEO ID TO COMPATIBLE FORMAT --- 
def convert_acs_to_nyc_geoid(geoid):
    """
    Convert ACS GEOID to NYC-style census tract GEOID.

    Parameters
    ----------
    geoid : str
        The GEOID string from ACS data, typically in the format '1500000US<state><county><tract>'.

    Returns
    -------
    str or float
        The NYC-style GEOID as a string, or np.nan if conversion is not possible.
    """
    if not isinstance(geoid, str) or not geoid.startswith("1500000US") or len(geoid) < 19:
        return np.nan
    cleaned_geoid = geoid[-12:]
    county_code = cleaned_geoid[2:5]
    tract_part = cleaned_geoid[5:]
    borough_prefix = {
        "061": "1",
        "005": "2",
        "047": "3",
        "081": "4",
        "085": "5",
    }.get(county_code)
    if borough_prefix is None:
        return np.nan
    return borough_prefix + tract_part.zfill(10)

# Apply cleaning directly from 'GEO_ID'
acs_merged['GEOID'] = acs_merged['GEOID'].apply(convert_acs_to_nyc_geoid)

In [ ]:
## -- EXPORT
# Export merged ACS data as CSV and pickle
acs_merged.to_csv(DATA_DIR / "processed/acs_merged.csv", index=False)
acs_merged.to_pickle(DATA_DIR / "processed/acs_merged.pkl")